In [12]:
import init
import pandas as pd
import tensorflow

pd.set_option('display.max_columns', 500)

In [34]:
def score(p1, p2):
    d = {1:0, 2:0}
    for _ in range(100):
        winner = init.main(100, 0, [p1, p2])
        d[winner] += 1

    win_rate = d[1]/100
    return win_rate

In [29]:
print(score(1, 'Default'))

0.504


In [30]:
print(score(0.8, 'Default'))

0.623


In [35]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense

# Define the architecture of the neural network
model = Sequential()
model.add(Dense(10, input_dim=1, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')

# Generate some training data
X_train = np.random.uniform(0, 1, size=(1000, 1))
y_train = np.array([score(x, 'Default') for x in X_train])

# Train the neural network
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Use the neural network to predict the input that maximizes the output of score()
def maximize_score():
    x = np.random.uniform(0, 1, size=(1, 1))
    for i in range(100):
        grad = model.predict(x) * (1 - model.predict(x)) * 2 * (score(x, 'Default') - 0.5)
        x += grad
        x = np.clip(x, 0, 1)
    return x[0][0]

Epoch 1/100
32/32 [==============================] - 1s 1ms/step - loss: 0.6933
Epoch 2/100
32/32 [==============================] - 0s 1ms/step - loss: 0.6932
Epoch 3/100
32/32 [==============================] - 0s 1ms/step - loss: 0.6932
Epoch 4/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6932
Epoch 5/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6932
Epoch 6/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6932
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: 0.6931
Epoch 8/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 9/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 10/100
32/32 [==============================] - 0s 1ms/step - loss: 0.6931
Epoch 11/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 12/100
32/32 [==============================] - 0s 2ms/step - loss: 0.6931
Epoch 13/100
32/32 [=================

In [37]:
print(score(.78, 'Default'))

0.59


In [36]:
input = [i/100 for i in range(1, 100)]
print(model.predict(input))

4/4 [==============================] - 0s 2ms/step
[[0.5025745 ]
 [0.5025745 ]
 [0.5025745 ]
 [0.5025745 ]
 [0.5025745 ]
 [0.5025745 ]
 [0.5025745 ]
 [0.5025745 ]
 [0.5025745 ]
 [0.5025745 ]
 [0.5025745 ]
 [0.5025923 ]
 [0.50262684]
 [0.50266135]
 [0.5026958 ]
 [0.5027303 ]
 [0.50276476]
 [0.50279933]
 [0.5028337 ]
 [0.50286824]
 [0.50290275]
 [0.5029372 ]
 [0.5029717 ]
 [0.50300616]
 [0.5030406 ]
 [0.5030751 ]
 [0.50310963]
 [0.5031441 ]
 [0.5031786 ]
 [0.50321305]
 [0.50324756]
 [0.503282  ]
 [0.50331646]
 [0.503351  ]
 [0.5033855 ]
 [0.50342   ]
 [0.5033441 ]
 [0.5032573 ]
 [0.50316644]
 [0.5030756 ]
 [0.50298476]
 [0.5028939 ]
 [0.5028031 ]
 [0.5027123 ]
 [0.5026215 ]
 [0.50253063]
 [0.5024398 ]
 [0.50234896]
 [0.5022581 ]
 [0.50216734]
 [0.5020765 ]
 [0.50198567]
 [0.50189483]
 [0.501804  ]
 [0.50171316]
 [0.5016223 ]
 [0.5015315 ]
 [0.5014407 ]
 [0.50134987]
 [0.5012591 ]
 [0.5011682 ]
 [0.50107735]
 [0.50098646]
 [0.50089574]
 [0.50080484]
 [0.50071406]
 [0.5006232 ]
 [0.5005323

In [27]:
print(maximize_score())

1/1 [==============================] - 0s 25ms/step
0.43276569528122966


In [45]:
import numpy as np

# Generate a range of inputs
X_test = np.linspace(0, 1, 100).reshape(-1, 1)

# Predict the output for each input
y_test = model.predict(X_test)

# Find the input that produces the highest output
x_best = X_test[np.argmax(y_test)]
y_best = np.max(y_test)

print("The input that produces the highest output is:", x_best[0])
print("The highest output is:", y_best)

4/4 [==============================] - 0s 2ms/step
The input that produces the highest output is: 0.3535353535353536
The highest output is: 0.50339764


In [100]:
def gradient_descent(p2, guess):
    def objective(x):
            return -score(x, p2)

    # Define the initial input, the learning rate, and the number of iterations
    x = guess
    learning_rate = 0.01
    num_iterations = 50

    # Define the beta parameters for the Adam optimizer
    beta1 = 0.9
    beta2 = 0.999

    # Define the initial values for the first and second moments
    m = 0
    v = 0

    iter_vals = []

    # Optimize the input using Adam optimization
    for i in range(num_iterations):
        if len(iter_vals) >= 2 and abs(iter_vals[-1] - iter_vals[-2]) < 0.0005:
             break

        # Compute the gradient of the objective function using numerical differentiation
        gradient = (objective(x + 0.001) - objective(x)) / 0.001
        # Change the sign of the gradient to maximize the objective function
        #gradient = -gradient
        # Update the first and second moments
        m = beta1 * m + (1 - beta1) * gradient
        v = beta2 * v + (1 - beta2) * gradient**2
        # Compute the bias-corrected first and second moments
        m_hat = m / (1 - beta1**(i+1))
        v_hat = v / (1 - beta2**(i+1))
        # Update the input using the Adam optimizer
        x = x - learning_rate * m_hat / (v_hat**(1/2) + 1e-8)

        iter_vals.append(x)

        print(x)

    return x, score(x, p2)

In [101]:
print(gradient_descent('Default', 0.8))

0.7900000000032259
0.7803978647490034
0.7720336223673754
0.7693033808419909
0.7661561155997165
0.7610619569174661
0.7543665013021877
0.750890637216982
0.7488750373711317
0.7478987895810434
0.7445145350214086
0.7417830564643616
0.7387802185298251
0.7365347932796241
0.7337734647097174
0.7315788680059311
0.7285448874853775
0.7266686619259888
0.724814687434162
0.7242500717627264
0.72180240755199
0.718161707984016
0.715780048867016
0.7133891351066971
0.7094800446657824
0.7083397147369838
0.7071016566104281
0.7071459040492122
(0.7071459040492122, 0.55)


In [104]:
def gradient_descent(p2, guess):
    def objective(x):
            return score(x, p2)

    # Define the initial input, the learning rate, and the number of iterations
    x = guess
    learning_rate = 0.01
    num_iterations = 50


    iter_vals = []

    # Optimize the input using Adam optimization
    for i in range(num_iterations):
        if len(iter_vals) >= 2 and abs(iter_vals[-1] - iter_vals[-2]) < 0.0005:
             break

        # Compute the gradient of the objective function using numerical differentiation
        gradient = (objective(x + 0.001) - objective(x)) / 0.001
       
        x += learning_rate * gradient

        iter_vals.append(x)

        print(x)

    return x, score(x, p2)

In [105]:
print(gradient_descent('Default', 0.8))

1.2800000000000005
1.0600000000000014
1.3200000000000005


KeyboardInterrupt: 